In [1]:
# set random seed
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Parameters

In [2]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.5 # polynomial degree
e = 0.5 # noise width

# Generate Data

In [4]:
from Data import data_generation
data_gen = data_generation()

#  ****** Data generation process is the same as SPO+ *********
# feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=42)
# split train test data
# from sklearn.model_selection import train_test_split
# x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

#  ****** Data generation process is the same as DDR *********
lower = 0
upper = 1
p = 5
d = 40
alpha = 1
mis = 4
n_epsilon = 1
W_star = data_gen.generate_truth("",lower, upper, p, d, version = 0) 
x_test, z_test_ori, c_test, x_train, z_train_ori, c_train, W_star = data_gen.generate_samples("",p, d, num_test, num_data, alpha, W_star, n_epsilon, mis, thres = 10, 
                        version = 1, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 

### SPO+ 

In [ ]:
from SPO_Plus import run_SPO_Shortest_Path
SPO_runner = run_SPO_Shortest_Path()
batch_size = 20
num_epochs = 30
arcs,loader_train,loader_test,cost_Oracle,cost_SPO = SPO_runner.run(x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)
                                                                run(DataPath_seed,x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)

TypeError: run() missing 1 required positional argument: 'is_run_SPO'

# OLS

In [22]:
from OLS import run_OLS_Shortest_Path
OLS_runner = run_OLS_Shortest_Path()
cost_OLS = OLS_runner.run(arcs,x_train,c_train,grid,loader_test)
print("Average OLS Cost = ",np.mean(cost_OLS))

Average OLS Cost =  6.742132800042629


# DDR

In [23]:
print("Average Oracle Cost = ",np.mean(cost_Oracle),"Std = ", np.std(cost_Oracle))
print("Average SPO Cost = ", np.mean(cost_SPO),"Std = ", np.std(cost_SPO))
print("Average OLS Cost = ", np.mean(cost_OLS),"Std = ", np.std(cost_OLS))

Average Oracle Cost =  5.942930629730225 Std =  1.379642731178901
Average SPO Cost =  6.77633664688468 Std =  1.6921440034748845
Average OLS Cost =  6.742132800042629 Std =  1.673809164608691


In [25]:
from DDR import run_DDR_Shortest_Path
DDR_runner = run_DDR_Shortest_Path()

mu_arr = np.arange(0.1,1,0.05)
lamb_arr = np.arange(0.1,1,0.05)
lamb_arr = [0.1,0.15,0.2,0.25,0.3,0.35,0.4]
minimum_value = 1000000000
for lamb in lamb_arr:
    print("======== lambda = ",lamb,"============")
    for mu in mu_arr:
        num_nodes = 25
        # w0_ddr_val,W_ddr_val = solve_DDR(lamb,mu,num_nodes,x_train,c_train)
        cost_DDR = DDR_runner.run(arcs,x_train, c_train, grid,loader_test,lamb,mu,num_nodes)
        if np.mean(cost_DDR) < minimum_value:
            minimum_value = np.mean(cost_DDR)
            print("lambda = ",lamb, ", mu = ",mu, ", Lowest Average DDR Cost = ",minimum_value, " Std = ",np.std(cost_DDR))
        # print("lambda = ",lamb, ", mu = ",mu, ",Average DDR Cost = ",np.mean(cost_DDR), " Std = ",np.std(cost_DDR))

======== lambda =  0.1 ============
lambda =  0.1 , mu =  0.1 , Lowerst Average DDR Cost =  6.744603146925568  Std =  1.6700484263066824
lambda =  0.1 , mu =  0.20000000000000004 , Lowerst Average DDR Cost =  6.744319741591811  Std =  1.6732742820144157
lambda =  0.1 , mu =  0.25000000000000006 , Lowerst Average DDR Cost =  6.738677947416901  Std =  1.6724724106727604
lambda =  0.1 , mu =  0.3500000000000001 , Lowerst Average DDR Cost =  6.7375347759723665  Std =  1.6740263761904977
======== lambda =  0.15 ============
======== lambda =  0.2 ============
======== lambda =  0.25 ============
======== lambda =  0.3 ============
======== lambda =  0.35 ============
======== lambda =  0.4 ============
